In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import tensorflow as tf

In [2]:
train_data=pd.read_csv('train.csv')

In [4]:
def NullClearner(value):
	if(isinstance(value, pd.Series) and (value.dtype in ['float64','int64'])):
		value.fillna(value.mean(),inplace=True)
		return value
	elif(isinstance(value, pd.Series)):
		value.fillna(value.mode()[0],inplace=True)
		return value
	else:return value
def handlemissing(X):
    x=X.columns.to_list()
    for i in x:
        X[i]=NullClearner(X[i])
    return X

In [5]:
train=train_data.drop('Property_ID',axis=1)

In [6]:
train=handlemissing(train)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39499 entries, 0 to 39498
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Property_Type           39499 non-null  object 
 1   Property_Area           39499 non-null  int64  
 2   Number_of_Windows       39499 non-null  float64
 3   Number_of_Doors         39499 non-null  int64  
 4   Furnishing              39499 non-null  object 
 5   Frequency_of_Powercuts  39499 non-null  float64
 6   Power_Backup            39499 non-null  object 
 7   Water_Supply            39499 non-null  object 
 8   Traffic_Density_Score   39499 non-null  float64
 9   Crime_Rate              39499 non-null  object 
 10  Dust_and_Noise          39499 non-null  object 
 11  Air_Quality_Index       39499 non-null  float64
 12  Neighborhood_Review     39499 non-null  float64
 13  Habitability_score      39499 non-null  float64
dtypes: float64(6), int64(2), object(6)
mem

In [67]:
X=train.drop('Habitability_score',axis=1)
Y=train['Habitability_score']

In [68]:
X=pd.get_dummies(X)

In [69]:
X.head()

,Property_Area,Number_of_Windows,Number_of_Doors,Frequency_of_Powercuts,Traffic_Density_Score,Air_Quality_Index,Neighborhood_Review,Property_Type_#R%$G&867,Property_Type_Apartment,Property_Type_Bungalow,...,Water_Supply_Once in a day - Evening,Water_Supply_Once in a day - Morning,Water_Supply_Once in two days,Crime_Rate_Slightly above average,Crime_Rate_Slightly below average,Crime_Rate_Well above average,Crime_Rate_Well below average,Dust_and_Noise_High,Dust_and_Noise_Low,Dust_and_Noise_Medium
0,106,3.923768,1,0.0,5.89,90.0,3.86,0,1,0,...,0,1,0,0,1,0,0,0,0,1
1,733,2.000000,2,1.0,4.37,96.0,3.55,0,1,0,...,1,0,0,0,0,0,1,0,0,1
2,737,4.000000,2,0.0,7.45,121.0,3.81,0,1,0,...,0,1,0,0,1,0,0,0,0,1
3,900,3.000000,2,2.0,6.16,100.0,1.34,0,1,0,...,0,1,0,0,0,1,0,0,0,1
4,2238,14.000000,6,0.0,5.46,116.0,4.77,0,0,1,...,0,0,0,0,0,0,1,0,0,1


In [70]:
from sklearn.preprocessing import scale

In [71]:
X=scale(X)

In [72]:
X

array([[-6.58239960e-01,  1.68597944e-16, -1.12833335e+00, ...,
        -2.97517308e-01, -2.35289403e-01,  3.92982429e-01],
       [-3.37238610e-01, -7.30356249e-01, -2.80838884e-01, ...,
        -2.97517308e-01, -2.35289403e-01,  3.92982429e-01],
       [-3.35190755e-01,  2.89413884e-02, -2.80838884e-01, ...,
        -2.97517308e-01, -2.35289403e-01,  3.92982429e-01],
       ...,
       [ 1.22271531e+00,  7.88239026e-01,  3.10913898e+00, ...,
        -2.97517308e-01, -2.35289403e-01,  3.92982429e-01],
       [-6.43618653e-02, -3.50707430e-01, -1.12833335e+00, ...,
        -2.97517308e-01, -2.35289403e-01,  3.92982429e-01],
       [-8.33045286e-02, -7.30356249e-01,  1.41415005e+00, ...,
         3.36114899e+00, -2.35289403e-01, -2.54464304e+00]])

In [ ]:
X.shape

In [91]:
def create_model(lr=0.01,act='relu'):  
    hidd1=tf.keras.layers.Dense(units=128,input_shape=[31],activity_regularizer=tf.keras.regularizers.l2(1e-5),activation=act)
    drop1=tf.keras.layers.Dropout(0.50)
    hidd2=tf.keras.layers.Dense(units=128,activity_regularizer=tf.keras.regularizers.l2(1e-5),activation=act)
    drop2=tf.keras.layers.Dropout(0.50)
    out=tf.keras.layers.Dense(units=1,activation=tf.nn.relu)

    model=tf.keras.Sequential([hidd1,drop1,hidd2,drop2,out])

    model.compile(optimizer=tf.optimizers.Adam(lr=lr),loss='mean_squared_error',metrics=['mse', "mape"])
    return model

In [45]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [46]:
X_train.shape,X_test.shape

((31599, 31), (7900, 31))

In [61]:
model.fit(X_train,Y_train,epochs=20)

Epoch 1/20
988/988 [==============================] - 3s 2ms/step - loss: 216.6790
Epoch 2/20
988/988 [==============================] - 2s 2ms/step - loss: 133.3533
Epoch 3/20
988/988 [==============================] - 2s 2ms/step - loss: 127.1914
Epoch 4/20
988/988 [==============================] - 2s 2ms/step - loss: 125.7159
Epoch 5/20
988/988 [==============================] - 2s 2ms/step - loss: 125.6664
Epoch 6/20
988/988 [==============================] - 2s 2ms/step - loss: 116.2884
Epoch 7/20
988/988 [==============================] - 2s 2ms/step - loss: 114.6740
Epoch 8/20
988/988 [==============================] - 2s 2ms/step - loss: 118.3242
Epoch 9/20
988/988 [==============================] - 2s 2ms/step - loss: 118.1092
Epoch 10/20
988/988 [==============================] - 2s 2ms/step - loss: 118.1790
Epoch 11/20
988/988 [==============================] - 2s 2ms/step - loss: 117.7014
Epoch 12/20
988/988 [==============================] - 2s 2ms/step - loss: 114.1511
E

In [116]:
from keras.wrappers.scikit_learn import KerasRegressor

In [125]:
model = KerasRegressor(build_fn = create_model,verbose = 0)

C:\Users\HP\AppData\Local\Temp\ipykernel_2528\1445120202.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn = create_model,verbose = 0)


In [126]:
params = {'act': ["relu", "tanh"],
          'batch_size': [16, 32, 64], 
          'epochs': [50, 100],
          'lr': [0.01, 0.001, 0.0001]}

In [127]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
random_search = RandomizedSearchCV(model,
                                   param_distributions = params,
                                   cv = KFold(3))

In [128]:
random_search_results = random_search.fit(X_train, Y_train)

C:\Users\HP\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [129]:
random_search_results

RandomizedSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x000001DF614B8BE0>,
                   param_distributions={'act': ['relu', 'tanh'],
                                        'batch_size': [16, 32, 64],
                                        'epochs': [50, 100],
                                        'lr': [0.01, 0.001, 0.0001]})

In [130]:
print("Best Score: ",
      random_search_results.best_score_,
      "and Best Params: ",
      random_search_results.best_params_)

Best Score:  -40.481100718180336 and Best Params:  {'lr': 0.01, 'epochs': 100, 'batch_size': 64, 'act': 'tanh'}


In [131]:
model=create_model(0.01,'tanh')

C:\Users\HP\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [132]:
model.fit(X_train,Y_train,epochs=100,batch_size=64)

Epoch 1/100
494/494 [==============================] - 10s 8ms/step - loss: 853.9304 - mse: 853.9285 - mape: 25.5041
Epoch 2/100
494/494 [==============================] - 3s 6ms/step - loss: 639.6377 - mse: 639.6359 - mape: 20.2789
Epoch 3/100
494/494 [==============================] - 4s 8ms/step - loss: 636.6580 - mse: 636.6558 - mape: 20.1139
Epoch 4/100
494/494 [==============================] - 4s 8ms/step - loss: 492.7952 - mse: 492.7927 - mape: 18.0494
Epoch 5/100
494/494 [==============================] - 3s 7ms/step - loss: 329.2512 - mse: 329.2489 - mape: 15.4990
Epoch 6/100
494/494 [==============================] - 4s 8ms/step - loss: 326.9136 - mse: 326.9113 - mape: 15.3235
Epoch 7/100
494/494 [==============================] - 4s 7ms/step - loss: 324.4286 - mse: 324.4265 - mape: 15.2014
Epoch 8/100
494/494 [==============================] - 3s 6ms/step - loss: 323.2165 - mse: 323.2143 - mape: 15.0946
Epoch 9/100
494/494 [==============================] - 2s 5ms/step - lo

494/494 [==============================] - 3s 5ms/step - loss: 52.3142 - mse: 52.3116 - mape: 8.7441
Epoch 73/100
494/494 [==============================] - 3s 5ms/step - loss: 52.0100 - mse: 52.0075 - mape: 8.7230
Epoch 74/100
494/494 [==============================] - 3s 7ms/step - loss: 52.3491 - mse: 52.3466 - mape: 8.7591
Epoch 75/100
494/494 [==============================] - 3s 6ms/step - loss: 51.7348 - mse: 51.7323 - mape: 8.7026
Epoch 76/100
494/494 [==============================] - 3s 5ms/step - loss: 51.8002 - mse: 51.7977 - mape: 8.7419
Epoch 77/100
494/494 [==============================] - 3s 5ms/step - loss: 51.7001 - mse: 51.6976 - mape: 8.6832
Epoch 78/100
494/494 [==============================] - 3s 6ms/step - loss: 51.8688 - mse: 51.8663 - mape: 8.7178
Epoch 79/100
494/494 [==============================] - 3s 6ms/step - loss: 51.5969 - mse: 51.5944 - mape: 8.6866
Epoch 80/100
494/494 [==============================] - 3s 5ms/step - loss: 51.3734 - mse: 51.3709 - 

In [133]:
model.evaluate(X_test,Y_test)

247/247 [==============================] - 2s 4ms/step - loss: 41.0572 - mse: 41.0548 - mape: 7.7154


[41.057247161865234, 41.05475616455078, 7.715404033660889]

In [112]:
model2=create_model(0.05,'tanh')

In [113]:
model2.fit(X_train,Y_train,epochs=20)

Epoch 1/20
988/988 [==============================] - 4s 3ms/step - loss: 795.3751 - mse: 795.3729 - mape: 24.8754
Epoch 2/20
988/988 [==============================] - 3s 3ms/step - loss: 705.8552 - mse: 705.8524 - mape: 22.5107
Epoch 3/20
988/988 [==============================] - 3s 3ms/step - loss: 579.0768 - mse: 579.0742 - mape: 20.3439
Epoch 4/20
988/988 [==============================] - 3s 3ms/step - loss: 323.0572 - mse: 323.0546 - mape: 15.3864
Epoch 5/20
988/988 [==============================] - 3s 3ms/step - loss: 146.8455 - mse: 146.8429 - mape: 12.7656
Epoch 6/20
988/988 [==============================] - 2s 2ms/step - loss: 84.4090 - mse: 84.4065 - mape: 11.4899
Epoch 7/20
988/988 [==============================] - 3s 3ms/step - loss: 84.6623 - mse: 84.6597 - mape: 11.5735
Epoch 8/20
988/988 [==============================] - 3s 3ms/step - loss: 83.1732 - mse: 83.1707 - mape: 11.4373
Epoch 9/20
988/988 [==============================] - 3s 3ms/step - loss: 82.6668 - ms